In [1]:
import torch
import numpy as np
import sys
sys.path.append("..")
import d2lzh_pytorch as d2l

# 获取和读取数据
这里继续使用Fashion-MNIST数据集。我们将使用多层感知机对图像进行分类。

In [2]:
batch_size = 256
train_iter, test_iter = d2l.load_data_fashion_mnist(batch_size)

# 定义模型参数
我们在`softmax`回归的从零开始实现里已经介绍了，Fashion-MNIST数据集中图像形状为 `28×28`，类别数为10。本节中我们依然使用长度为 `28×28=784` 的向量表示每一张图像。因此，输入个数为784，输出个数为10。实验中，我们设超参数隐藏单元个数为256。

In [3]:
num_inputs, num_outputs, num_hiddens = 784, 10, 256

W1 = torch.tensor(np.random.normal(0, 0.01, (num_inputs, num_hiddens)), dtype=torch.float)
b1 = torch.zeros(num_hiddens, dtype=torch.float)
W2 = torch.tensor(np.random.normal(0, 0.01, (num_hiddens, num_outputs)), dtype=torch.float)
b2 = torch.zeros(num_outputs, dtype=torch.float)

params = [W1, b1, W2, b2]
for param in params:
    param.requires_grad_(requires_grad=True)

# 定义激活函数
这里我们使用基础的`max`函数来实现ReLU，而非直接调用`relu`函数。

In [4]:
def relu(X):
    return torch.max(input=X, other=torch.tensor(0.0))

# 定义模型
同softmax回归一样，我们通过`view`函数将每张原始图像改成长度为`num_inputs`的向量。然后我们实现上一节中多层感知机的计算表达式。

In [5]:
def net(X):
    X = X.view((-1, num_inputs))
    H = relu(torch.matmul(X, W1) + b1)
    return torch.matmul(H, W2) + b2

# 定义损失函数
为了得到更好的数值稳定性，我们直接使用PyTorch提供的包括softmax运算和交叉熵损失计算的函数。

In [6]:
loss = torch.nn.CrossEntropyLoss()

# 训练模型
训练多层感知机的步骤和3.6节中训练softmax回归的步骤没什么区别。我们直接调用`d2lzh_pytorch`包中的`train_ch3`函数，它的实现已经在3.6节里介绍过。我们在这里设超参数迭代周期数为5，学习率为100.0。

In [7]:
num_epoches, lr = 6, 100.0
d2l.train_ch3(net, train_iter, test_iter, loss, num_epoches, batch_size, params, lr)

epoch 1, loss 0.0031, train acc 0.713, test acc 0.786
epoch 2, loss 0.0019, train acc 0.823, test acc 0.822
epoch 3, loss 0.0016, train acc 0.845, test acc 0.828
epoch 4, loss 0.0016, train acc 0.854, test acc 0.845
epoch 5, loss 0.0014, train acc 0.864, test acc 0.843
epoch 6, loss 0.0014, train acc 0.870, test acc 0.842


# 小结
+ 可以通过手动定义模型及其参数来实现简单的多层感知机。
+ 当多层感知机的层数较多时，本节的实现方法会显得较烦琐，例如在定义模型参数的时候。